In [1]:
from random import choice, sample, randrange, random, shuffle
from string import ascii_lowercase
import numpy as np

### Defining the fitness function

$fitness(w) = \sum^{nºguesses}_{i=1}(|correct(g_i,w) - c_i|+|misplaced(g_i,w) - m_i|)$

Where $w$ is a word for which we will evaluate the fitness, $g_i$ is the guess played on turn $i$, $c_i$ is the number of correct values obtained in the guess $i$, $correct(g,r)$ return the number of correct positions in a Wordle game provided that $g$ is the attempt and $w$ is the secret word, $misplaced(g,w)$ returns the number of misplaced letters given that $g$ is the attempt and $w$ is the secret word, $m_i$ is the number of misplaced letters obtained in the attempt $i$. 

In [6]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def check_words(input_word ,target):
    input_word = input_word.lower()
    target = target.lower()
    sequence = ['⬜']*5
    selected = [False]*5
    for i in range(0,5):
        if input_word[i] == target[i]:
            sequence[i] = '🟩'
            selected[i] = True
    for i in range(0,5):
        if input_word[i] != target[i]:
            indexes = find(target, input_word[i])
            for index in indexes:
                if not selected[index]:
                    selected[index] = True
                    sequence[i] = '🟨'
                    break
    aux = ""
    return aux.join(sequence)

In [5]:
def fitness(word, game_state):
    sum = 0
    for guess, result in game_state:
        match = result.count('🟩')
        misplaced =  result.count('🟨')
        result = check_words(guess, word)
        pos_match = result.count('🟩')
        pos_misplaced = result.count('🟨')
        sum += abs(pos_match - match) + abs(pos_misplaced - misplaced)
    return (sum + const*len(word)*(self.guess_count_-1))

### Defining mutation
Mutation occurs when a random letter of the word is changed.

In [7]:
def mutate(word, prob=0.03):
    if random() < prob:
        word = list(word)
        word[randrange(len(word))] = choice(ascii_lowercase)
        word = ''.join(word)
    return word

### Defining permutation
Permutation occurs when two letters from a word are switched

In [8]:
def permute(word, prob=0.03):
    if random() < prob:
        idx1, idx2 = sample(range(len(word)), 2)
        word = list(word)
        word[idx1], word[idx2] = word[idx2], word[idx1]
        word = ''.join(word)
    return word

### Defining inversion
Inversion occurs when two positions are randomly selected and the sequence of letters between is inverted.

In [12]:
def invert(word, prob=0.02):
    if random() < prob:
        idx1, idx2 = sample(range(len(word)), 2)
        idx1, idx2 = min(idx1,idx2), max(idx1,idx2)
        word = list(word)
        word[idx1:idx2] = reversed(word[idx1:idx2])
        word = ''.join(word)
    return word

### Defining crossover

In [11]:
def crossover(parent1, parent2, prob=0.5):
    child1, child2 = parent1, parent2
    if random() < prob:
        split = randrange(1, len(parent1)-1)
        child1 = parent1[:split] + parent2[split:]
        child2 = parent2[:split] + parent1[split:]
    return [child1, child2]